## Prepare ad media data info

Google ad media data are saved on wesmedia1

In [1]:
import pandas as pd
import os
import hashlib

### 1. Get video info

In [2]:
video_path = '/data/1/wesmediafowler/projects/AdMedia/Google/video'

In [3]:
filepath = []
filename = []

for root, dirs, files in os.walk(video_path):
    for file in files:
        if file.endswith(".mp4"):
            filename.append(file)
            filepath.append(os.path.join(root,file))

In [5]:
vid = pd.DataFrame(columns=['filename', 'filepath'])

In [6]:
vid['filename'] = filename
vid['filepath'] = filepath

#### Get video checksum

In [7]:
def hash_bytestr_iter(bytesiter, hasher, ashexstr=False):
    for block in bytesiter:
        hasher.update(block)
    return hasher.hexdigest() if ashexstr else hasher.hexdigest()

def file_as_blockiter(afile, blocksize=65536):
    with afile:
        block = afile.read(blocksize)
        while len(block) > 0:
            yield block
            block = afile.read(blocksize)

In [9]:
sha = [(fname, hash_bytestr_iter(file_as_blockiter(open(fname, 'rb')), hashlib.sha256()))
    for fname in filepath]

In [10]:
checksum = pd.DataFrame(sha, columns=['filepath', 'checksum_sha256'])

In [12]:
video = vid.merge(checksum, on='filepath', how='left', indicator=True)

In [13]:
video._merge.value_counts()

both          25932
left_only         0
right_only        0
Name: _merge, dtype: int64

In [15]:
del video['_merge']

In [16]:
video.head()

,filename,filepath,checksum_sha256
0,nIAa2wx0y2Q.mp4,/data/1/wesmediafowler/projects/AdMedia/Google...,3eb256475d44d124d8d671b15460066e767a87efaea137...
1,ZL_4kcX7kIM.mp4,/data/1/wesmediafowler/projects/AdMedia/Google...,16270b71732f94fd4a9bc2169f76cf94010d4e0bacbe1d...
2,QDt2eixQ-7E.mp4,/data/1/wesmediafowler/projects/AdMedia/Google...,2f5617ad11a6c8900a106261d9e56d5eb34c750c159312...
3,wAE3Adxg8z4.mp4,/data/1/wesmediafowler/projects/AdMedia/Google...,bc61eec68d979c23eacdee58c5df66a915c669d93283e8...
4,31E4r62s34c.mp4,/data/1/wesmediafowler/projects/AdMedia/Google...,d34b9f98fd5a1084181969b3ee61ebcd9f40bc79b2593f...


In [18]:
video.to_csv("google2022_video_info.csv", index=False)